In [1]:
import pandas as pd
import numpy as np

<br>
read data_1  -- albums names and sales(!) (scraped from wikipedia)<br>
read lyricDataset -- albums names and lyrics(!) (API from Genius) 

In [2]:
df_1 = pd.read_csv('data_1.csv')
columns = ['Artist', 'Title', 'Album', 'Lyric_views', 'Release_date', 'Featured_art',
           '#_words', 'Lyrics']
df_2 = pd.read_csv('lyricDataset.csv', names = columns)

<br>
drop unnecessary columns

In [3]:
lyrics = df_2.drop(['Title', 'Release_date', 'Featured_art', '#_words'], axis = 1)

<br>
leave in dataframe "lyrics" only those rows which has albums from dataframe df_1 with sales

In [4]:
albums = np.array(df_1['Album'])
lyrics = lyrics[lyrics['Album'].isin(albums)]

<br>
now leave in dataframe "sales" only rows with albums from dataframe "lyrics"

In [5]:
almums = np.array(lyrics['Album'])
sales = df_1[df_1['Album'].isin(albums)]

In [6]:
sales.reset_index(drop=True, inplace=True)
lyrics.reset_index(drop=True, inplace=True)

In [7]:
sales['Album'].nunique()

168

In [8]:
lyrics['Album'].nunique()

90

<br>
take lyrics from "lyrics" and prepare it for analysis (take out punctuation and put in lowercase)

In [9]:
str_array = lyrics['Lyrics'].to_numpy()

In [10]:
length = len(str_array)

In [11]:
import string
for i in range(length):
    str_array[i] = str_array[i].translate(string.punctuation).replace('+', ' ').lower()

<br>
Run sentiment_analysis for each string (lyrics) and store result in 'sent_result'

In [12]:
sent_result = np.empty(length)

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#note: depending on how you installed (e.g., using source code download versus pip install), you may need to import like this:
#from vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
for i in range(length):
    sent_result[i] = analyzer.polarity_scores(str_array[i])['compound']

In [14]:
sent_result

array([ 0.9973, -0.5122,  0.9897,  0.9224,  0.9986,  0.9938,  0.9923,
       -0.9722, -0.9789,  0.9991,  0.9853,  0.9976,  0.8748,  0.9963,
        0.9992,  0.9995, -0.9971,  0.8759,  0.9562,  0.9807,  0.6966,
        0.9825,  0.9901,  0.9955,  0.9988, -0.9934,  0.9904,  0.9971,
        0.9863, -0.8246,  0.9376,  0.9928, -0.9825, -0.9844,  0.9903,
       -0.9547,  0.9877,  0.9821, -0.9114,  0.1672, -0.9924, -0.971 ,
        0.5269,  0.9954, -0.9349, -0.9879, -0.9197, -0.999 , -0.9894,
       -0.9993,  0.8901,  0.9947, -0.9939, -0.3773,  0.9945, -0.9822,
        0.9579,  0.9974,  0.2669,  0.9932, -0.2826,  0.9958,  0.9463,
       -0.9859, -0.7554,  0.9926,  0.9716,  0.9983,  0.9921,  0.9324,
        0.986 ,  0.9039,  0.9995,  0.988 ,  0.9936,  0.9793,  0.9952,
       -0.5499,  0.9936,  0.9618,  0.9751,  0.9951,  0.999 , -0.9522,
       -0.9969, -0.9858, -0.9828, -0.9996,  0.9233, -0.9823, -0.9978,
        0.8872, -0.9942, -0.979 , -0.9983, -0.9994, -0.9995,  0.9994,
       -0.9997, -0.9

In [15]:
x = pd.Series(sent_result)

In [16]:
lyrics = lyrics.assign(Sent = x)

In [17]:
lyrics

,Artist,Album,Lyric_views,Lyrics,Sent
0,Ariana Grande,Sweetener,3034628,you you love it how i move you you love it ho...,0.9973
1,Ariana Grande,Dangerous Woman,2634773,i've been here all night (ariana) i've been he...,-0.5122
2,Ariana Grande,Sweetener,2628876,right now i'm in a state of mind i wanna be i...,0.9897
3,Ariana Grande,Sweetener,1824759,lacigam gnihtemos od ot thgin laiceps ruoy s't...,0.9224
4,Ariana Grande,Sweetener,1138842,mmm yeah yuh i thought you into my life (wh...,0.9986
...,...,...,...,...,...
315,Justin Timberlake,Man of the Woods,137506,ain't no need of stopping girl yeah there ai...,-0.9719
316,Travis Scott,Birds in the Trap Sing McKnight,4067071,yeah 7:30 in the night yeah ooh-oooh ooh i ...,0.9965
317,Travis Scott,Birds in the Trap Sing McKnight,1847257,that coca i just poured an 8 in a liter throw...,-0.9854
318,Travis Scott,Birds in the Trap Sing McKnight,1134501,hmm hmm meh meh mmm meh meh meh hmm hmm...,0.9938


In [18]:
x = lyrics.groupby('Album')['Sent'].mean()

In [19]:
x

Album
...And Justice for All   -0.269633
1000 Forms of Fear       -0.008975
1989                     -0.309300
24K Magic                 0.675413
4:44                      0.279240
                            ...   
Views                     0.993533
Wrapped in Red            0.990400
Wu-Tang Forever          -0.582620
X                        -0.202040
Yeezus                   -0.992500
Name: Sent, Length: 90, dtype: float64